In [ ]:
import pandas as pd
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, LSTM, CuDNNLSTM
import math

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [ ]:
n_cols = range(1, 65)
data = pd.read_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\final_dataset\\dataset.csv', usecols= n_cols)
data = data.sample(n=5000, random_state=1)

In [ ]:
data_train = data.drop(['Y_CAT_STORNO_TOCH', 'DELTA_STORNO_FROMREZ',
                  'DELTA_STORNO_TOCH',  'SIF_HOTELA', 'OBRISANO_DA', 'Y_CAT_STORNO_FROMREZ'], axis=1) 
data_train = data_train.sort_values(by=['GODINA'], ascending=False)

In [ ]:
Xtrain = data_train.drop('STORNO_DA', axis=1)
Ytrain = data_train.STORNO_DA 

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
opt = keras.optimizers.Adam(lr=0.001, decay=1e-6)

In [ ]:
cvscores= []
f = folds.get_n_splits()
with tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=16)) as sess:
    backend.set_session(sess)
    for train, test in folds.split(Xtrain, Ytrain):
        print(f)
        print('*************************************************************************************************')
        X_train, X_test = Xtrain.iloc[train], Xtrain.iloc[test]
        Y_train, Y_test = Ytrain.iloc[train], Ytrain.iloc[test]

        #DNN
        model = Sequential([
        Dense(512, activation='relu', input_shape=(57,)),
        Dropout(0.2),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid'),
        ]) 
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(X_train, Y_train, epochs=20)

        scores = model.evaluate(X_test, Y_test)
        cvscores.append(('DNN', scores[1]))

        #CNN
        X_train = X_train.values.tolist()
        X_test = X_test.values.tolist()
        X_train = np.reshape(X_train, (1, 57, np.shape(X_train)[0])).T
        X_test = np.reshape(X_test, (1, 57, np.shape(X_test)[0])).T

        model = Sequential([
            Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(57,1)),
            MaxPooling1D(pool_size=2),
            Conv1D(filters=3, kernel_size=2, activation='relu'),
            MaxPooling1D(pool_size=2),
            Dropout(0.2),
            Flatten(),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dropout(0.2),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(X_train, Y_train, epochs=20)

        scores = model.evaluate(X_test, Y_test)
        cvscores.append(('CNN', scores[1]))

        #RNN
        model = Sequential([
            LSTM(256, input_shape=(57, 1), activation='relu', return_sequences=True),
            Dropout(0.2),
            LSTM(128, activation='relu'),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dropout(0.2),
            Dense(16, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(X_train, Y_train, epochs=1)

        scores = model.evaluate(X_test, Y_test)
        cvscores.append(('RNN', scores[1]))

        f -= 1

In [ ]:
scores_df = pd.DataFrame(cvscores, columns=['model', 'score'])

In [ ]:
df = []
for m in ['DNN', 'CNN', 'RNN']:
    s = np.mean(scores_df['score'][scores_df['model']==m])
    std = np.std(scores_df['score'][scores_df['model']==m])
    df.append((m,s,std))

In [ ]:
scores = pd.DataFrame(df, columns=['model', 'score', 'std'])
scores